# &#129504; Understanding attributes

Based on the PyCon US 2022 &#x1F40D; talk "Reuven M. Lerner: Understanding attributes (Or: They're not nearly as boring as you think!)" 

&#x1F3AC; https://www.youtube.com/watch?v=Tn1wLsj7Bys&list=PL2Uw4_HvXqvYeXy8ab7iRHjA-9HiYhRQl&index=51

## &#x1F4D6; Contents  

1. [Getting started](#start) 
2. [Variables vs. attributes](#vari) 
3. [Reading from attributes](#read) 
4. [Setting attributes](#set) 
5. [Extending person](#person) 
6. [Class attributes](#class) <br>
6.1 [Quiz time!](#quiz) <br>
6.2 [Methods are attributes](#methods) <br>
6.3 [Classes are file-less modules](#file_less) <br>
6.4 [Static variables](#static) 
7.   [ICPO](#icpo) <br>
7.1 [Employees](#employees) <br>
7.2 [Print](#print) <br>
7.3 [Operator overloading](#ops) 
8. [Method rewriting](#rewriting) <br>
8.1 [Descriptors](#desc) <br>
8.2 [get](#get) <br>
8.3 [Methods are descriptors](#meth_r_desc) <br>
8.4 [Partial functions](#partial)
9. [Wrapping up](#wrap) 

# &#127939; &#8205; Getting started <a id="start"></a>

In [1]:
# let's assign to a variable

x = 100

What is happening here?

> We are **not** putting the value of `100` in the memory location named `x`

> We **are** saying that the name `x` should refer to the integer object `100`

This is best illustrated using

> [Python tutor](https://pythontutor.com/) &#x1F4F9;

from which we see that the global variable `x` refers to an integer object `100`. So far, so good &#128077; 

In [6]:
# let's create a class 
# a do nothing class

class MyClass:
    pass

x = MyClass    # create an instance, assign to x
x.y = 100      # assign 100 to x.y

What am I doing here?

> We are **not** assigning to `x` ; we **are** assigning to `x.y`

> We **are** creating an attribute on `x` ; `y` is **not** a variable

&#x1F4F9;  `x` refers to an instance of `MyClass`, and in that class we have an attribute `y` with a value of `100`.

# &#x1F93C; Variables vs. attributes <a id="vari"></a>

> Every object in Python &#x1F40D; has attributes

> Attributes are like a private dictionary &#x1F4D8; but we use `.` vs. `[]` to set or retrieve them

> The attribute exists on the object &#129513; that the variable refers to -- **not** the variable itself

So, while we could colloquially say:

"`x` has an attribute `y`",

this is in fact false &#x274C;

"`x` is the variable referring to the object, and that object has an attribute `y`" &#x2705; 

# &#x1F4DA; Reading from attributes <a id="read"></a>

In [7]:
# we can retrieve from attributes easily

import sys    # create the global variable sys which refers to the module object
sys.version   # version is an attribute on the sys variable

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

And we receive the version of Python that we're running &#x1F3C3; 

In [8]:
str.upper('abc')    # str is a built-in which refers to the string class
                    # upper is a method on str
                    # once I retrieve the method (function) object, use parentheses to invoke it and receive the result

'ABC'

In [9]:
import random
random.randint(0,100)    # random is a module (variable)
                         # randint is a method
                         # retrieve method object and invoke it

3

> Attributes can contain **any** Python object e.g data &#x1F4CA; or funcs &#x1F57A; &#127926; 

# Setting attributes 🩹<a id="set"></a>

With rare exceptions - typically relating to built-ins written in C - we can set **any** attribute on **any** object we want. 

In [10]:
# assuming I have an object that's
# NOT one of the built-in types

x.y = 100

We have added an attribute `y` to the object referred to by `x`. If `y` already existed, we would've replaced vs. added it.

If this sounds similar to a dictionary &#x1F4D8; Guess what? That's because it is.

In [11]:
# to emphasise the point

sys.version = "4.20"
sys.version

'4.20'

Often, we set attributes without thinking about it. For example, the `__init__` method is designed to set attributes!

In [12]:
class Person:
    def __init__(self, name):    # self = instance created 
        self.name = name         # name = local variable containing value        
                                
p1 = Person('name1')
p2 = Person('name2')

What is happening here? &#128173;

> We take the value `name` and assign to `self.name` i.e. add an attribute to that instance. Now the object has that attribute, referred to by `self` within `__init__` but a different reference or variable otherwise e.g. `p1` or `p2`

Hint: use [Python tutor](https://pythontutor.com/) &#x1F4F9; 

What is `__init__` ?

> It is **not** the constructor method &#128679;

> It is invoked **after** the new object has been created to add **attributes** to the object &#129513;

# &#x1F465; Extending `Person` <a id="person"></a>

Let's keep track of the population as we use `Person` to create people &#x1F6B6;

In [13]:
# Idea: a global variable

population = 0    # create a global variable

class Person:
    def __init__(self, name):
        population +=1    # add 1 to population each time a new instance is created 
        self.name = name
        
    def greet (self):
        return f'Hello, {self.name}!'
    
    
print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {population=}')

Before, population=0


UnboundLocalError: cannot access local variable 'population' where it is not associated with a value

Why the error? &#x274C; Because

> Within functions, variable assignments are local &#x1f3e0; and `population` is assigned **inside** `__init__`

So, when we run `__init__`, Python &#x1F40D; says

> "I need 1 + the current value of...&#128165;"

In [14]:
# How to fix this?
# Use `global`

population = 0   

class Person:
    def __init__(self, name):
        global population
        population +=1    
        self.name = name
        
    def greet (self):
        return f'Hello, {self.name}!'
    
    
print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {population=}')

Before, population=0
After, population=2


How does this work? &#x2705; 

> `global` says: "Hey Python, when you compile this function, **don't** record `population` as a local variable. Instead, assign to the global variable."

&#x1F197; Okay but unnecessary use of global variables is poor practice. Can we do better?

# &#8205; &#127891; Class attributes <a id="class"></a>

In [15]:
# Yes! Set an attribute on the Person class

class Person:
    def __init__(self, name):
        
        Person.population +=1
        self.name = name
        
    def greet (self):
        return f'Hello, {self.name}!'
    
Person.population = 0    # set population as an attribute on Person
    
    
print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {population=}')

print(p1.greet())
print(p2.greet())

Before, population=2
After, population=2
Hello, name1!
Hello, name2!


Unfortunately this works...the solution is correct &#x2705; but ugly &#x1F922; 

# 🃏 Quiz time! <a id="quiz"></a>

In [16]:
# What will this code print?

print('A')    # 1. duh

class Person:
    print('B')    # 2. class body
    def  __init__(self, name):
        print('C')    # 5. function body
        self.name = name
    print('D')    # 3. class body
print('E')    # 4. class body


p1 = Person('name1')
p2 = Person('name2')

A
B
D
E
C
C


What's going on here? &#x1f914; Comparing func vs. class defs will help:

> A function body **doesn't** execute when it's defined

> A class body **must** execute when it's defined



&#x1F44C; Understand. But there's something weird about `def`...

Normally, `def` does two things:

> Creates a function object &#129513;

> Assigns the object to a variable- the function's name &#x1F520; 

But here, `def` is defining `__init__`. 

What is `__init__`?

# 👩‍🍳 Methods are attributes <a id="methods"></a>

`__init__` is an attribute. Methods are attributes!

Just like `population`, `__init__` is an attribute on the class. 

> Hint: use [Python tutor](https://pythontutor.com/) &#x1F4F9; 

&#x1F197; Okay but we still don't understand how `def` can define an attribute, when usually it defines a variable?

# &#8205; &#127891; Classes are file-less modules <a id="file_less"></a>

We can think of classes as modules minus the files &#x1F4C1;

> Functions defined **in** a class body are class **attributes** &#x1F516;   

> **Inside** of the class, they look lke variables 🪑

> **Outside**, they look like attributes on the class &#x1F333;

This is just like modules!

> Functions or variables defined in a module file &#x1F4C1; are: global variables **inside** the file or attributes on the module object **oustide** the file.

So, returning to our example:

In [17]:
# Class attribute, nice edition

class Person:
    population = 0    # attribute NOT variable definition!
    
    def __init__(self, name):
        
        Person.population +=1
        self.name = name
        
    def greet (self):
        return f'Hello, {self.name}!'
    
    
print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {population=}')

print(p1.greet())
print(p2.greet())

Before, population=2
After, population=2
Hello, name1!
Hello, name2!


&#x1F44C; Understand. But why don't we call it `person.population`?

> `Person` is defined **after** the class body has run &#x1F3C3;

# &#x26A1; Static variables <a id="static"></a>

Sometimes, class attributes are called "static variables". 

**Don't** do this &#x1F6AB; 

Why?

> Static variables are **shared** across the class and instances, whereas a class attribute &#x1F516; is a name which exists **only** on the class

> Python &#x1F40D; does **not** have the concept of a shared attribute. However, many different attributes (and variables) can refer to the same object.

In [18]:
# Let's see the difference

class Person:
    population = 0    
    
    def __init__(self, name):
        
        Person.population +=1
        self.name = name
    
    
print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {Person.population=}')
print(f'After, {p1.population=}')
print(f'After, {p2.population=}')

Before, population=2
After, Person.population=2
After, p1.population=2
After, p2.population=2


Uh oh...it worked &#128169; And we got the same values. Maybe they are shared...&#x1F914;

# ICPO 🔎 <a id="icpo"></a>

Nope! We've encountered ICPO - the attribute lookup rule in Python. This is how Python &#x1F40D; looks for attributes -- in every instance.

> **I**nstance : does this attribute exist on this instance? If yes, stop &#x26D4; else...

> **C**lass : does this instance's class have this attribute? If yes, stop &#x26D4; else...

> **P**arent : does this instance's class's parent have this attribute? If yes, stop &#x26D4; else...

> **O**bject : does this instance's class's parent's object have this attribute?

In [19]:
# Let's do a walkthrough

class Person:
    population = 0    
    
    def __init__(self, name):
        
        Person.population +=1
        self.name = name
    
    
print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {Person.population=}')    # does person (instance) have the attribute population? Yes! Return 2.
print(f'After, {p1.population=}')        # does p1 (instance) have the attribute population? No.
print(f'After, {p2.population=}')        # okay, does person (class) have the attribute population? Yes! Return 2.

Before, population=2
After, Person.population=2
After, p1.population=2
After, p2.population=2


In [20]:
# Another example

class Person:
    
    def __init__(self, name):
        self.name = name
        
    def greet (self):
        return f'Hello, {self.name}!'
    

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())    # does p1 (instance) have the attribute greet? No.
print(p2.greet())    # does person (class) have the attribute greet? Yes! We get back method object and execute it

Hello, name1!
Hello, name2!


Methods are **class** attributes but we normally call them via the **instance**. ICPO lookup explains how this is possible.

# 👩‍💼 Employees <a id="employees"></a>

In [21]:
# let's define a new class: Employee
# Employee should have 2 attributes: name and ID number

class Employee:
    
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number
        
    def greet (self):
        return f'Hello, {self.name}!'
    

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())    
print(e2.greet())  

Hello, emp1!
Hello, emp2!


This works but...wouldn't it be better if we used **inheritance**?

&#x1F926;

In [22]:
# let's try again, except with inheritance

class Employee(Person):    # inserts Person as parent into the ICPO attribute search
    
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number
        
    def greet (self):
        return f'Hello, {self.name}!'
    

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())    # does e1 (instance) have the attribute greet? No.
print(e2.greet())    # does Employee (class) have the attribute greet? Yes! We get back method object and execute it

Hello, emp1!
Hello, emp2!


We have a solution using inheritance. But observe that the `greet` methods in `Person` and `Employee` are the same. This is definitely **not** dry &#127964;&#65039;

In [23]:
# Get rid of the greet method in Employee

class Employee(Person):    
    
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number
    

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())    # does e1 (instance) have the attribute greet? No.
print(e2.greet())    # does Employee (class) have the attribute greet? No.
                     # does Person (parent) have the attribute greet? Yes! We get back method object and execute it

Hello, emp1!
Hello, emp2!


&#x1F192; Cool but why do we need to set `self.name` in `Employee.__init__`? Couldn't we just rely on `Person.__init__`?

In [24]:
# Get rid of self.name in Employee

class Employee(Person):    
    
    def __init__(self, name, id_number):

        self.id_number = id_number
    

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())    # does e1 (instance) have the attribute greet? No.
print(e2.greet())    # does Employee (class) have the attribute greet? No.
                     # does Person (parent) have the attribute greet? Yes! We get back method object and execute it

AttributeError: 'Employee' object has no attribute 'name'

No. Why the error? &#x274C; 

> The `name` attribute was **never** set because we never got to `Person.__init__`

When a new instance of `Employee` is created, we ask:

> **I**nstance: does `e1` have `__init__`? No &#x26D4;

> **C**lass: does `Employee` have `__init__`? Yes &#x2705;

`Employee.__init__` runs, setting `self.id_number`. Because `Employee__init__` is found, `Person.__init__` never runs and `name` isn't added!

In [25]:
# solution 1
# explicitly call Person.__init__

class Employee(Person):    
    
    def __init__(self, name, id_number):
        Person.__init__(self, name)
        self.id_number = id_number
    

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())   
print(e2.greet())

Hello, emp1!
Hello, emp2!


This works but it's better to use `super`  &#x1f9b8; &#x200d; 

What is `super`?

> `Super` &#x1f9b8; &#x200d; is a proxy which says "I will figure out who we can run `__init__` on". It looks up the chain 	&#9939;

In [26]:
# solution 2
# use super()

class Employee(Person):    
    
    def __init__(self, name, id_number):
        super().__init__(name)
        self.id_number = id_number
    

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())   
print(e2.greet())

Hello, emp1!
Hello, emp2!


# 🖨️ Print <a id="print"></a>

What happens when I print `p1` or `e1`?

In [27]:
print(p1)
print(e1)

Why so ugly? &#x1F922; 

> Calling `print` invokes `str`, which calls the `__str__` method

For `p1`

> **I**nstance: does `p1` have `__str__`? No &#x26D4;

> **C**lass: does `p1`'s class (`Person`) have `__str__`? No &#x26D4;

> **O**bject: does object have `__str__`? Yes! &#x2705;

So we get the default (&#x1F922;) implementation of `__str__`

Similarly for `e1`

> **I**nstance: does `e1` have `__str__`? No &#x26D4;

> **C**lass: does `e1`'s class (`Employee`) have `__str__`? No &#x26D4;

> **P**arent: does `e1`'s class's parent (`Person`) have `__str__`? No &#x26D4;

> **O**bject: does object have `__str__`? Yes! &#x2705;

# &#x2797;  Operator overloading <a id="ops"></a>

Attribute lookup &#128064; is the reason that operator overloading works.

If we define `__str__` in a class, we stop ICPO at an earlier stage than **O**bject.

# &#x1F4DD; Method rewriting <a id="rewriting"></a>

In [28]:
# the normal way to invoke a method

s = 'zoe'
s.upper()    # ask for upper att. on instance 
             # doesn't expect an argument

'ZOE'

In [29]:
# an alternative way, which yields the same result

str.upper(s)    # ask for upper att. on class
                # expects instance as an argument

'ZOE'

We know:

1. Methods are stored in class attributes

2. We can retrieve a class attribute via the class or the instance

But we're getting **different** behaviour depending on how we retrieve it!

In [30]:
# let's ask
Person.greet

<function __main__.Person.greet(self)>

In [31]:
p1.greet

<bound method Person.greet of <__main__.Person object at 0x0000019D7DFA9F90>>

What's going on? Descriptors &#128213;

# &#128213; Descriptors <a id="desc"></a>

Normally, when I retrieve a class attribute, I recieve the object stored in the attribute. 

But
> If the attribute's class has a `__get__` method, then the result of `__get__` is returned instead!

In [32]:
# Example

class LoudNumber:
    
    def __init__(self, n):
        print(f'In LoudNumber.__init__, {n=}')
        self.n = n 
        
    def __get__(self, *args):
        print(f'In LoudNumber.__get__, {self.n=}')
        return self.n

If I assign an instance of `LoudNumber` to a variable, nothing happens. But if I assign it to a **class attribute**, magic happens &#129497;&#8205;&#9794;&#65039; &#10024; 

In [33]:
# assign it to a class attribute

class Person:
    age = LoudNumber(25)

In LoudNumber.__init__, n=25


This output since, when a class is defined, the definition is run &#x1F3C3; 

In [34]:
# create an instance of Person
p = Person()

What's happening here? &#128173;

> We have `Person`, which refers to the person class. It has an attribute, `age`, taking the value an instance of `LoudNumber` (25)

Hint: use [Python tutor](https://pythontutor.com/) &#x1F4F9; 

In [37]:
# Retreive the value (class attribute)
# 1. via the instance
# 2. via the class

p.age

In LoudNumber.__get__, self.n=25


25

In [38]:
Person.age

In LoudNumber.__get__, self.n=25


25

Observe

> We **didn't** use parentheses. This means the method shouldn't have been invoked - but it was &#129497;&#8205;&#9794;&#65039; &#10024; 

# 📥 `__get__` <a id="get"></a>

What are the parameters of `__get__` ?

> `obj` : from what instance is this attribute being retrieved?

> `objtype` : in what class is this attribute stored?

In [39]:
# let's plug them in

class LoudNumber:
    
    def __init__(self, n):
        print(f'In LoudNumber.__init__, {n=}')
        self.n = n 
        
    def __get__(self, obj, objtype):
        print(f'Also : {obj=}, {objtype=}')
        print(f'In LoudNumber.__get__, {self.n=}')
        return self.n

In [40]:
# and see the difference

class Person:
    age = LoudNumber(25)

In LoudNumber.__init__, n=25


In [41]:
p = Person()    # class att. defined on Person (objtype)
p.age           # retrieve via p, the instance (obj)

Also : obj=<__main__.Person object at 0x0000019D7EC5A710>, objtype=<class '__main__.Person'>
In LoudNumber.__get__, self.n=25


25

In [42]:
Person.age    # class att. defined on Person (objtype)
              # retrieve via Person, the class (obj)

Also : obj=None, objtype=<class '__main__.Person'>
In LoudNumber.__get__, self.n=25


25

# 👨‍🍳 Methods are descriptors <a id="meth_r_desc"></a>

When we retrieve a method via the **class**, Python returns the original function we defined. It knows because `obj` is set to None. 

Thus we need to supply an instance as the first argument.

In [43]:
# Example

s = 'wild world'
str.upper(s)

'WILD WORLD'

In contrast, when we retrieve a method via an **instance**, Python does magic rewriting &#129497;&#8205;&#9794;&#65039; &#x270D; 

Specifically, it turns `obj` - the instance on which we ran the method - into the first argument (`self`)  

How does Python &#x1F40D; achieve this? Partial functions!

# &#x1F313; Partial functions <a id="partial"></a>

A partial function is a function with some arguments "pre-loaded" &#128299;

In [44]:
# Example

from functools import partial

def add(a, b):
    return a + b

add3 = partial(add, 3)    # preload first arg 3 onto add

add3(10)

13

This is the **same** situation as ours. 

When we invoke a method via the instance, Python uses `partial` to automatically assign (or bind &#129657;) that instance to the method. 


In [45]:
# Remember this?

class Person:
    
    def __init__(self, name):
        self.name = name
        
    def greet (self):
        return f'Hello, {self.name}!'
    

p1 = Person('name1')
p2 = Person('name2')

Person.greet

<function __main__.Person.greet(self)>

In [46]:
p1.greet

<bound method Person.greet of <__main__.Person object at 0x0000019D7DF28990>>

Observe &#128064;

```python
bound method
```

Python &#x1F40D; is literally telling us what's happening! But it's hard to understand if we don't first understand attributes.

**Where's the OG (original `greet`)?**

Moved and stored on another class attribute, `__dict__`

```python
{ "method name" : "original function" }
```



In summary

> Ask for a method via the class: `obj` is **None** and method descriptor returns og &#x1F4B3; function from `__dict__`

> Ask for a method via the instance: `obj` is **not** None. Python &#x1F40D; creates a partial &#x1F313;   with instance + function and returns them together. This is our method &#128104;&#8205;&#127859;

# &#x1F381; Wrapping up <a id="wrap"></a>

> Attributes give us: class attributes, inheritance and method rewriting &#129497;&#8205;&#9794;&#65039; &#x270D; 
  
> Python &#x1F40D; looks for our attribute via ICPO &#128064;

> Attributes can be descriptors &#128213;

> Descriptors allow us to use methods via the instance or class &#127891;

Never think of `a.b` the same again!

# The End!

In [1]:
from IPython.display import Image
Image(url='https://media.giphy.com/media/m8w1PMQtDsWgtt6OLE/giphy.gif')